In [1]:
from datetime import datetime
from sklearn.manifold import TSNE
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.init as init
from torch.autograd import Variable
import argparse
import copy
import numpy as np
import time
import os
from PIL import Image
from utils.dataloader import *
from utils import new_transforms



    

In [2]:
exp_name="breast_sub_basal_vs_all_7layers_tr"
test_val="valid"
im_size="299"
model="7layers"
tlog="/scratch/sb3923/logs/breast_sub_basal_vs_all_7layers_tr.log"
 
#breast subtypes
root_dir="/beegfs/sb3923/DeepCancer/alldata/BreastSubtypes_basal_vs_all/BreastTilesSorted/"
num_class=2 
tile_dict_path="/beegfs/sb3923/DeepCancer/alldata/BreastSubtypes_basal_vs_all/Breast_FileMappingDict.p"
#--val=${test_val} --imgSize=${im_size} --model_type=${model}"
nexp="/scratch/sb3923/experiments/breast_sub_basal_vs_all_7layers_tr"
out="/scratch/sb3923/logs/$valid"
    
PathToEpoch="$/scratch/sb3923/experiments/breast_sub_basal_vs_all_7layers_tr/checkpoints/"
Cmodel="epoch_1.pth"

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--experiment', default='', help="name of experiment to test")
parser.add_argument('--model', default='', help="name of model to test")
parser.add_argument('--root_dir', type=str, default='<ROOT_PATH><CANCER_TYPE>TilesSorted/', help='Data directory .../dataTilesSorted/')
parser.add_argument('--num_class', type=int, default=2, help='number of classes ')
parser.add_argument('--tile_dict_path', type=str, default='"<ROOT_PATH><CANCER_TYPE>_FileMappingDict.p', help='Tile dictinory path')
parser.add_argument('--val', type=str, default='test', help='validation set')
parser.add_argument('--train_log', type=str, default='/gpfs/scratch/bilals01/test-repo/logs/exp6_train.log', help='point to the log file created from the training')
parser.add_argument('--imgSize', type=int, default=299, help='the height / width of the image to network')
parser.add_argument('--model_type',type=str,  default='PathCNN', help='choose the model to train with: PathCNN, alexnet,vgg16')
parser.add_argument('--nc', type=int, default=3, help='input image channels (+ concatenated info channels if metadata = True)')
parser.add_argument('--ngpu'  , type=int, default=1, help='number of GPUs to use')


In [3]:

#opt = parser.parse_args()
nc = 3
root_dir = root_dir
num_classes = 2
tile_dict_path = tile_dict_path
tl_file = tlog
test_val = test_val
imgSize = 299
ngpu = 1
test_val = 'test'


imgSize = 299

transform = transforms.Compose([new_transforms.Resize((imgSize,imgSize)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#use Tissuedata2 for ownsampled data, for test use the whole data
test_data = TissueData(root_dir, test_val, train_log=tl_file, transform = transform, metadata=False)
#test_data = TissueData(root_dir, test_val, train_log='', transform = transform, metadata=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=8)

#os.chdir("/scratch/sb3923/deep-cancer/tsne_figures/")
#pickle.dump( test_data.filenames, open( "test_data.p", "wb" ) )

classes = test_data.classes

class_to_idx = test_data.class_to_idx
 
print('Class encoding:')
print(class_to_idx)

print('after tissuedata')




Loading from: Basal
number of samples: 10114
Loading from: non-Basal
number of samples: 69738
Class encoding:
{'non-Basal': 1, 'Basal': 0}
after tissuedata


In [4]:
def get_tile_probability(tile_path):

    """
    Returns an array of probabilities for each class given a tile

    @param tile_path: Filepath to the tile
    @return: A ndarray of class probabilities for that tile
    """

    # Some tiles are empty with no path, return nan
    if tile_path == '':
        return np.full(num_classes, np.nan)

    tile_path = root_dir + tile_path

    with open(tile_path, 'rb') as f:
        with Image.open(f) as img:
            img = img.convert('RGB')

    # Model expects a 4D tensor, unsqueeze first dimension
    img = transform(img).unsqueeze(0)
    img = img.cuda()

    # Turn output into probabilities with softmax
    var_img = Variable(img, volatile=True)
    output = F.softmax(model(var_img)[0]).data.squeeze(0) 
    return output.cpu().numpy()
    #original: 
    #var_img = Variable(img, volatile=True)
    #output = F.softmax(model(var_img)).data.squeeze(0)

    #return output.cpu().numpy()


def get_tile_probability2(tile_path):
    """
    Returns an array of probabilities for each class given a tile

    @param tile_path: Filepath to the tile
    @return: A ndarray of class probabilities for that tile
    """
    # Some tiles are empty with no path, return nan
    if tile_path == '':
        return np.full(1024, np.nan)

    tile_path = root_dir + tile_path

    with open(tile_path, 'rb') as f:
        with Image.open(f) as img:
            img = img.convert('RGB')
    # Model expects a 4D tensor, unsqueeze first dimension
    img = transform(img).unsqueeze(0)
    img = img.cuda()
    # Turn output into probabilities with softmax
    var_img = Variable(img, volatile=True)
    

    viz = (model(var_img)[1]).squeeze(0)#torch.FloatTensor of size 1x5184 
    return viz.data.cpu().numpy() #numpy.ndarray



with open(tile_dict_path, 'rb') as f:
    tile_dict = pickle.load(f)


def aggregate(file_list, method):

    """
    Given a list of files, return scores for each class according to the
    method and labels for those files.

    @param file_list: A list of file paths to do predictions on
    @param method: 'average' - returns the average probability score across
                               all tiles for that file
                   'max' - predicts each tile to be the class of the maximum
                           score, and returns the proportion of tiles for
                           each class

    @return: a ndarray of class probabilities for all files in the list
             a ndarray of the labels

    """

    model.eval()
    predictions = []
    true_labels = []
    last_layer = []
    file_name = []

    for file in file_list:
        tile_paths, label = tile_dict[file]

        folder = classes[label]

        def add_folder(tile_path):
            if tile_path == '':
                return ''
            else:
                return folder + '/' + tile_path

        # Add the folder for the class name in front
        add_folder_v = np.vectorize(add_folder)
        tile_paths0 = add_folder_v(tile_paths)

        # Get the probability array for the file
        prob_v= np.vectorize(get_tile_probability, otypes=[np.ndarray])
        probabilities = prob_v(tile_paths0)
        
        tile_paths1 = add_folder_v(tile_paths)
        
        prob_v2 = np.vectorize(get_tile_probability2, otypes=[np.ndarray])
        lastlayer = prob_v2(tile_paths1)

        probabilities = np.stack(probabilities.flat)
        prediction = np.nanmean(probabilities, axis = 0)

        tile_label = np.argmax(probabilities,axis=1)
        #last layer
        lastlayer = np.stack(lastlayer.flat)
        a = lastlayer[np.ix_(label == tile_label),:]

        if (np.squeeze(a)).ndim>1:
            lastlayerweights = np.nanmean(np.squeeze(a), axis = 0)
            #lastlayer = np.stack(lastlayer.flat)
            #lastlayerweights = np.nanmean(lastlayer, axis = 0)

            predictions.append(prediction)
            true_labels.append(label)
            last_layer.append(lastlayerweights)
            file_name.append(file)

    return np.array(predictions), np.array(true_labels), np.array(last_layer),np.array(file_name)
    #return np.array(true_labels), np.array(last_layer)



class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, pool, **kwargs):
        super(BasicConv2d, self).__init__()

        self.pool = pool
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
        self.relu = nn.LeakyReLU()
        
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = self.conv(x)

        if self.pool:
            x = F.max_pool2d(x, 2)
        
        x = self.relu(x)
        x = self.bn(x)
        x = self.dropout(x)
        return x

# Define model
class cancer_CNN(nn.Module):
    def __init__(self, nc, imgSize, ngpu):
        super(cancer_CNN, self).__init__()
        self.nc = nc
        self.imgSize = imgSize
        self.ngpu = ngpu
        #self.data = 'all'
        self.conv1 = BasicConv2d(nc, 16, False, kernel_size=5, padding=1, stride=2, bias=True)
        self.conv2 = BasicConv2d(16, 32, False, kernel_size=3, bias=True)
        self.conv3 = BasicConv2d(32, 64, True, kernel_size=3, padding=1, bias=True)
        self.conv4 = BasicConv2d(64, 64, True, kernel_size=3, padding=1, bias=True)
        self.conv5 = BasicConv2d(64, 128, True, kernel_size=3, padding=1, bias=True)
        self.conv6 = BasicConv2d(128, 64, True, kernel_size=3, padding=1, bias=True)
        self.linear = nn.Linear(5184, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        llw=x
        x = self.linear(x)
        return x, llw

class cancer_CNN_7layers(nn.Module):
     def __init__(self, nc, imgSize, ngpu):
         super(cancer_CNN_7layers, self).__init__()
         self.nc = nc
         self.imgSize = imgSize
         self.ngpu = ngpu
         #self.data = opt.data
         self.conv1 = BasicConv2d(nc, 16, False, kernel_size=5, padding=1, stride=2, bias=True)
         self.conv2 = BasicConv2d(16, 32, False, kernel_size=3, bias=True)
         self.conv3 = BasicConv2d(32, 64, True, kernel_size=3, padding=1, bias=True)
         self.conv4 = BasicConv2d(64, 64, True, kernel_size=3, padding=1, bias=True)
         self.conv5 = BasicConv2d(64, 128, True, kernel_size=3, padding=1, bias=True)
         self.conv6 = BasicConv2d(128, 64, True, kernel_size=3, padding=1, bias=True)
         self.conv7 = BasicConv2d(64, 64, True, kernel_size=3, padding=1, bias=True)
         self.linear = nn.Linear(1024, num_classes)
 
     def forward(self, x):
         x = self.conv1(x)
         x = self.conv2(x)
         x = self.conv3(x)
         x = self.conv4(x)
         x = self.conv5(x)
         x = self.conv6(x)
         x = self.conv7(x)
         x = x.view(x.size(0), -1)
         llw=x
         x = self.linear(x)
         return x, llw


model = cancer_CNN_7layers(nc, imgSize, ngpu)

model.cuda()

model_path = nexp + '/checkpoints/' + Cmodel
state_dict = torch.load(model_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state_dict)



In [5]:
predictions, labels, fw_lastlayer, file_names = aggregate(test_data.filenames, method='average')
print('------------------------------------------------------')
print('last-layer')
print(fw_lastlayer)



/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/ipykernel/__main__.py:124: RuntimeWarning: Mean of empty slice


------------------------------------------------------
last-layer
[[-0.16158456 -0.16323298 -0.20049755 ...,  0.04818501  0.02766555
   0.01556775]
 [-0.24808421 -0.26689126 -0.26869134 ...,  0.05977428  0.05279158
   0.05143663]
 [-0.20628169 -0.23520254 -0.22418504 ..., -0.04810501 -0.05702516
  -0.05158457]
 ..., 
 [ 0.14700307  0.15369099  0.1491881  ..., -0.20098462 -0.177492
  -0.17318917]
 [ 0.1439863   0.17981805  0.16721435 ..., -0.09529176 -0.13053024
  -0.13672796]
 [ 0.18447607  0.18488079  0.1739374  ...,  0.00434389  0.00074014
   0.02845178]]


In [7]:
nexp.split('/')[-1]

'breast_sub_basal_vs_all_7layers_tr'

In [10]:
! pwd

/scratch/sb3923/PathCNN/tsne_data


In [12]:
#os.chdir("tsne_data/")
 
exp_name = nexp.split('/')[-1]
pickle.dump( labels, open( "finalWs_{0}_{1}_{2}.p".format(exp_name,test_val,Cmodel), "wb" ) )



In [ ]:
finalWs = fw_lastlayer
os.chdir("tsne_data/")
pickle.dump( finalWs, open( "finalWs_{0}_{1}_{2}.p".format(opt.experiment,opt.val,opt.model), "wb" ) )
pickle.dump( predictions, open( "predictions_{0}_{1}_{2}.p".format(opt.experiment,opt.val,opt.model), "wb" ) )
pickle.dump( labels, open( "labels_{0}_{1}_{2}.p".format(opt.experiment,opt.val,opt.model), "wb" ) )
pickle.dump( file_names, open( "file_names_{0}_{1}_{2}.p".format(opt.experiment,opt.val,opt.model), "wb" ) )


In [13]:
! pwd

/scratch/sb3923/PathCNN/tsne_data


In [18]:
pickle_in = open("predictions_breast_sub_basal_vs_all_7layers_tr_test_epoch_1.pth.p","rb")
dat = pickle.load(pickle_in)
dat

array([[ 0.33290977,  0.66709023],
       [ 0.59178376,  0.40821624],
       [ 0.40041099,  0.59958901],
       [ 0.41988901,  0.58011099],
       [ 0.31546959,  0.68453041],
       [ 0.53517736,  0.46482264],
       [ 0.09809893,  0.90190107],
       [ 0.26828116,  0.73171884],
       [ 0.25845903,  0.74154097],
       [ 0.58529355,  0.41470645],
       [ 0.32410907,  0.67589093],
       [ 0.31708972,  0.68291028],
       [ 0.25304629,  0.74695371],
       [ 0.74000518,  0.25999482],
       [ 0.18208446,  0.81791554],
       [ 0.3602923 ,  0.6397077 ],
       [ 0.17681244,  0.82318756],
       [ 0.60194043,  0.39805957],
       [ 0.64769227,  0.35230773],
       [ 0.34080859,  0.65919141],
       [ 0.41917276,  0.58082724],
       [ 0.35803225,  0.64196775],
       [ 0.35005047,  0.64994953],
       [ 0.32500136,  0.67499864],
       [ 0.71681158,  0.28318842],
       [ 0.10473872,  0.89526128],
       [ 0.34547851,  0.65452148],
       [ 0.16110428,  0.83889572],
       [ 0.20798294,